In [1]:
from pyspark.sql.functions import concat, lit, col,expr,split, to_timestamp,date_format,round
from pyspark.sql.types import IntegerType, DoubleType, StringType, DateType


from zoneinfo import ZoneInfo
from datetime import datetime,timedelta

StatementMeta(, b37492cf-f6a2-49b0-8430-192078247837, 3, Finished, Available, Finished)

In [2]:
egypt_tz = ZoneInfo('Africa/Cairo')
yesterday = datetime.now(egypt_tz) - timedelta(days=1)
table_name = f"youtube_trending_{yesterday.strftime('%Y_%m_%d')}"

StatementMeta(, b37492cf-f6a2-49b0-8430-192078247837, 4, Finished, Available, Finished)

In [3]:
df=spark.read.table(table_name)
display(df)

StatementMeta(, b37492cf-f6a2-49b0-8430-192078247837, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2a91c793-c3d5-400f-842a-428e4d1a10cb)

In [4]:
new_added_columns = (
    df.withColumn(
        "video_link",
        concat(lit("https://youtube.com/watch?v="), col("id"))
    )
    .withColumn(
        "channel_link",
        concat(lit("https://www.youtube.com/channel/"), col("channelId"))
    )
    .withColumn(
        "engagement_rate",
        round(
            expr("((cast(likeCount as double) + cast(commentCount as double)) / cast(viewCount as double)) * 100"),
            2
        )
    )
    .withColumn(
        "engagement_to_subscribers_ratio",
        round(
            expr("((cast(likeCount as double) + cast(commentCount as double)) / cast(subscriberCount as double)) * 100"),
            2
        )
    )
)

StatementMeta(, b37492cf-f6a2-49b0-8430-192078247837, 6, Finished, Available, Finished)

In [5]:
new_added_columns = new_added_columns.withColumn("published_at_ts", to_timestamp("publishedAt"))

# Then, split into separate date and time columns
new_added_columns = new_added_columns.withColumn("published_date", col("published_at_ts").cast("date")) \
       .withColumn("published_hour", date_format("published_at_ts", "HH").cast("int"))



StatementMeta(, b37492cf-f6a2-49b0-8430-192078247837, 7, Finished, Available, Finished)

In [6]:
final_df=new_added_columns.drop('published_at_ts','published_time','publishedAt','id','channelID')
display(final_df)

StatementMeta(, b37492cf-f6a2-49b0-8430-192078247837, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9c513fda-3708-43b5-82dc-216ffbae27e4)

In [7]:
final_df = final_df.withColumn("channelTitle", col("channelTitle").cast(StringType())) \
       .withColumn("title", col("title").cast(StringType())) \
       .withColumn("viewCount", col("viewCount").cast(IntegerType())) \
       .withColumn("likeCount", col("likeCount").cast(IntegerType())) \
       .withColumn("commentCount", col("commentCount").cast(IntegerType())) \
       .withColumn("engagement_rate", col("engagement_rate").cast(DoubleType())) \
       .withColumn("video_link", col("video_link").cast(StringType())) \
       .withColumn("channel_link", col("channel_link").cast(StringType())) \
       .withColumn("published_date", col("published_date").cast(DateType())) \
       .withColumn("published_hour",col("published_hour").cast(IntegerType()))\
       .withColumn("url",col("url").cast(StringType()))\
       .withColumn("subscriberCount",col("subscriberCount").cast(IntegerType()))\
       .withColumn("engagement_to_subscribers_ratio",col("engagement_to_subscribers_ratio").cast(DoubleType()))
display(final_df)

StatementMeta(, b37492cf-f6a2-49b0-8430-192078247837, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 24f0c6f2-5eb6-48fe-9c3b-d3baf9388dfe)

In [8]:
final_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"youtube_trending_{yesterday.strftime('%Y_%m_%d')}")

StatementMeta(, b37492cf-f6a2-49b0-8430-192078247837, 10, Finished, Available, Finished)